# Why

A simple tutorial to build a MARTINI membrane with or without a protein using the INSANE tool. Following the building of the membrane, a short minimization run will be done. 

# Prerequisite & notes

All files are located in the insane_tutorial subdirectory. 

You will need:
* The insane.py script (provided)
* The martinize.py script (provided)
* DSSP (provided)
* The MARTINI force-field files (provided)
* A .pdb of your membrane protein (provided)
* VMD, to visualize the structures ! http://www.ks.uiuc.edu/Research/vmd/
* If you want to further do simulations/minimization, GROMACS http://www.gromacs.org/

Insane is published here:
* T.A. Wassenaar, H.I. Ingólfsson, R.A. Böckmann, D.P. Tieleman, S.J. Marrink. Computational lipidomics with insane: a versatile tool for generating custom membranes for molecular simulations. JCTC, 11:2144–2155, doi:2015.10.1021/acs.jctc.5b00209

# Pick a membrane protein !

A few notes here. 

* Since cristallographic data can miss the structure of loops, sometimes the protein isn't whole in the .pdb - so make sure that your protein is WHOLE. Or reconstruct the missing parts.
* No cofactor or weird stuff attached to your protein. 
* For your first steps, pick a protein with only alpha helices. No beta strands (Technical reason: the side-chain of amino-acids in the MARTINI model sometimes don't behave like they should, and although it's not usually affecting what you might look at, I simply suggest to stay away from this at first. Also, there's a fix for this on the MARTINI's website if you're interested).

In this tutorial, we will use the structure of ... the human Aquaporin 4 (PDB ID: 3GD8) ! Just a monomer. Because a tetramer needs a few adjustments. As a small modification to the original PDB, I just removed everything that wasn't protein from the file.

# How-to

## Step 0 - Open a terminal or use this notebook

## Step 1 - Martinize the protein (skip this if you want membrane only)
A single command is enough for this, using the *martinize.py* script.

In [8]:
%%bash

# First make all scripts executable
chmod u+x scripts/*.py

./scripts/martinize.py -v -f working_folder/3gd8.pdb -o working_folder/3gd8.top -x working_folder/3gd8_init.pdb -n working_folder/3gd8_bindex -nmap working_folder/3gd8_map -dssp working_folder/dssp-2.2.1/mkdssp -ff martini22 -p Backbone -elastic -ef 500 -el 0.41 -eu 0.9

# Also, move the output to the appropriate folder
mv Protein_A.itp working_folder
mv *.ssd working_folder


	There you are. One MARTINI. Shaken, not stirred.


He was white and shaken, like a dry martini. 
                                                               --P. G. Wodehouse 



INFO       MARTINIZE, script version 2.6
INFO       If you use this script please cite:
INFO       de Jong et al., J. Chem. Theory Comput., 2013, DOI:10.1021/ct300646g
INFO       Chain termini will be charged
INFO       Residues at chain brakes will not be charged
INFO       The martini22 forcefield will be used.
INFO       Local elastic bonds will be used for extended regions.
INFO       Position restraints will be generated.
WARNING    Position restraints are only enabled if -DPOSRES is set in the MDP file
INFO       Read input structure from file.
INFO       Input structure is a PDB file.
INFO       Found 1 chains:
INFO          1:   A (Protein), 1654 atoms in 223 residues.
INFO       Total size of the system: 223 residues.
DEBUG      New version of DSSP; Executing 'insane_tutorial/dssp-2.2.1/mkdssp -i /dev/stdin -o chain_A.ssd'
DEBUG      DSSP determined secondary structure:
 HHHHHHHHHHHHHHHHHHHHHHHT  TTTTTS     HHHHHHHHHHHHHHHHHHHHHHH    SHHHHHHHHHTTSS HHHHHHHHHHHHHHHHHHHHHHHHHS H

A few explanations on this command (you can skip this). -f is the input file. -o, -x, -n and -nmap are output files and you only need -x. -dssp is the location of the secondary structure predictor DSSP. -ff is the version of the force-field (martini22) and -p, -elastic, -ef, -el, -eu are options relative to the elastic network. As a reminder, the elastic network is used to replace the degrees of liberty lost because of the coarse graining, ie we lost the fine structure, atom by atom, and therefore we lose hydrogen bonds necessary to keep the secondary structure together.

## Step 2 - INSANE ! Build the membrane.
As before, a single command is enough, using the *insane.py* script. We will later need to use it another time.

Quick options:
* -x, -y and -z define the dimensions of the box (membrane on the x and y dimensions, z for water).
* -rectangular and -center are good all the time. Add -orient if it doesn't come from the OPM.
* -d 8 is good like this, only modify it if you put more than one protein in the membrane.
* -l defines lipid proportions ! This is what we want. Basically, all the lipids that you can find in *insane.py*: DPPG, DPPC, POPC, DOPC, DLPE, DPPE, POPE, DOPE, POPG, DOPG, POPS, DOPS, GMO, inositols like DPPI, POPU, PIPI, PAPI, PUPI, POP1, POP2, POP3, glycolipids like DPG1, DXG1, PNG1, XNG1, DPG3, cardiolipins like CDL2, CXL2, CDL0, mycolic acids, sterols ... Everything defined in the force-field file martini_v2.0_lipids_all_201506.itp !

We will use a funny membrane for this tutorial, but you can just modify it. 

In [9]:
%%bash

# With protein
./scripts/insane.py -charge 0 -salt 0.15 -x 20 -y 20 -z 15 -sol W -f working_folder/3gd8_init.pdb -o working_folder/3gd8_membrane.gro -p working_folder/3gd8_membrane.top -pbc rectangular -d 8 -center -l CHOL:4 -l FPSG:1 -l PAPI:1 -l DPG1:3 -l PIPI:2 -l POPG:4 -l POPE:8

# Without protein
./scripts/insane.py -charge 0 -salt 0.15 -x 20 -y 20 -z 15 -sol W -o working_folder/membrane.gro -p working_folder/membrane.top -pbc rectangular -l CHOL:4 -l FPSG:1 -l PAPI:1 -l DPG1:3 -l PIPI:2 -l POPG:4 -l POPE:8

; X: 20.000 (26 bins) Y: 20.000 (26 bins) in upper leaflet
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in lower leaflet
; 658 lipids in upper leaflet, 658 lipids in lower leaflet
; NDX Solute 1 470
; Charge of protein: 0.000000
; NDX Membrane 471 18142
; Charge of membrane: -228.000000
; Total charge: -228.000000
; NDX Solvent 18143 48618
; NDX System 1 48618
; "I mean, the good stuff is just INSANE" --Julia Ormond
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in upper leaflet
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in lower leaflet
; 676 lipids in upper leaflet, 676 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 18202
; Charge of membrane: -234.000000
; Total charge: -234.000000
; NDX Solvent 18203 48581
; NDX System 1 48581
; "I mean, the good stuff is just INSANE" --Julia Ormond


Now you have your membrane ! You can either leave now or stay for ~20 more minutes to equilibrate your membrane.

## Step 3 - Clean the topology, give the right charge to insane, add antifreeze particles and clean the topology again
That's a lot. But not that much in the end !
### Clean the topology
*insane.py* put a single *#include* statement at the beginning of the .top files. That's actually no good and we need to include **all** the force-field files. This is easy enough to do by **hand**, since this tutorial isn't supposed to include much code. For the purpose of this tutorial, I made a small python script that will do just that, and we'll use it here.

In [10]:
%%bash

./scripts/clean_include_topology.py -f working_folder/3gd8_membrane.top -o working_folder/3gd8_membrane_clean.top
./scripts/clean_include_topology.py -f working_folder/membrane.top -o working_folder/membrane_clean.top

### Check the charge of the membrane
Why do we want a neutral system in the end ? Because non neutral systems aren't physical. To check the current charge of our system, we'll use GROMACS and do as if we wanted to do a minimization. It will tell us the charge of the system. 

Of course, this seems like a hack. And it is. Ideally, we would pick this information from the topology, but that would require some code, some classes to handle what's in the .itp. Ain't nobody got time for that.

In [11]:
%%bash

gmx grompp -f params/minim.mdp -o working_folder/checking_charge.tpr -p working_folder/3gd8_membrane_clean.top -c working_folder/3gd8_membrane.gro

gmx grompp -f params/minim.mdp -o working_folder/checking_charge2.tpr -p working_folder/membrane_clean.top -c working_folder/membrane.gro

Analysing residue names:
There are:   223    Protein residues
There are: 31784      Other residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 6 Mb of data
Analysing residue names:
There are: 31725      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 6 Mb of data


                  :-) GROMACS - gmx grompp, VERSION 5.1.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University

See the **NOTE** ? It tells us the charge of the membrane. This is also the step where you'll notice any problem with the topology (like, do the atom names in the .gro match those in the .top ?). 

**Our membrane with protein therefore has a charge of -624 and our membrane without protein has a charge of -642.**

### Rebuild the membrane and clean the topology (again)
Simply reusing the same commands from earlier, except we now know which charge to input INSANE.

In [12]:
%%bash

# With protein
./scripts/insane.py -charge -624 -salt 0.15 -x 20 -y 20 -z 15 -sol W -f working_folder/3gd8_init.pdb -o working_folder/3gd8_membrane.gro -p working_folder/3gd8_membrane.top -pbc rectangular -d 8 -center -l CHOL:4 -l FPSG:1 -l PAPI:1 -l DPG1:3 -l PIPI:2 -l POPG:4 -l POPE:8

# Without protein
./scripts/insane.py -charge -642 -salt 0.15 -x 20 -y 20 -z 15 -sol W -o working_folder/membrane.gro -p working_folder/membrane.top -pbc rectangular -l CHOL:4 -l FPSG:1 -l PAPI:1 -l DPG1:3 -l PIPI:2 -l POPG:4 -l POPE:8

# Cleaning topology
./scripts/clean_include_topology.py -f working_folder/3gd8_membrane.top -o working_folder/3gd8_membrane_clean.top
./scripts/clean_include_topology.py -f working_folder/membrane.top -o working_folder/membrane_clean.top

; X: 20.000 (26 bins) Y: 20.000 (26 bins) in upper leaflet
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in lower leaflet
; 658 lipids in upper leaflet, 658 lipids in lower leaflet
; NDX Solute 1 470
; Charge of protein: 0.000000
; NDX Membrane 471 18142
; Charge of membrane: -228.000000
; Total charge: -228.000000
; NDX Solvent 18143 48700
; NDX System 1 48700
; "I mean, the good stuff is just INSANE" --Julia Ormond
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in upper leaflet
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in lower leaflet
; 676 lipids in upper leaflet, 676 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 18202
; Charge of membrane: -234.000000
; Total charge: -234.000000
; NDX Solvent 18203 48581
; NDX System 1 48581
; "I mean, the good stuff is just INSANE" --Julia Ormond


### -- Optional step: add antifreeze particles --
The Coarse Grain nature of our system has consequences on water. Water's low fusion point is a consequence of the many hydrogen bonds constantly forming and breaking. When we put 4 water molecules into one, we lose these interactions and therefore our CG water will freeze at a higher temperature than normal water.

Water doesn't always freeze in CG systems, which is why this step is optional. Moreover, as we'll use a high temperature to speed everything up (325K), the system probably won't freeze. **But** you still have to visually check if the water freezes if you don't put antifreeze particles (it's quite obvious visually). 

We usually just transform 10% of our water beads (W) into antifreeze beads (WF), simply by replacing them in the .gro and adding the right amount in the .top. This can be done by **hand** if you're lazy, or you can use the small script below, which comes with absolutely no guarantee.

In [13]:
%%bash

./scripts/add_antifreeze.py -c working_folder/3gd8_membrane.gro -co working_folder/3gd8_membrane_with_antifreeze.gro -p working_folder/3gd8_membrane_clean.top -po working_folder/3gd8_membrane_with_antifreeze.top

./scripts/add_antifreeze.py -c working_folder/membrane.gro -co working_folder/membrane_with_antifreeze.gro -p working_folder/membrane_clean.top -po working_folder/membrane_with_antifreeze.top

And now test with grompp if everything is fine ~~ Basically, pray, but it should be fine. Remember, this guide comes with no guarantee.

In [14]:
%%bash

gmx grompp -f params/minim.mdp -o working_folder/prot_em.tpr -p working_folder/3gd8_membrane_with_antifreeze.top -c working_folder/3gd8_membrane_with_antifreeze.gro

gmx grompp -f params/minim.mdp -o working_folder/memb_em.tpr -p working_folder/membrane_with_antifreeze.top -c working_folder/membrane_with_antifreeze.gro

# Small cleanup
rm \#mdout*
cd working_folder
rm \#checking*
rm checking*
rm \#prot*
rm \#memb*

Analysing residue names:
There are:   223    Protein residues
There are: 31866      Other residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 6 Mb of data
Analysing residue names:
There are: 31725      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 6 Mb of data


                  :-) GROMACS - gmx grompp, VERSION 5.1.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University

## Step 4 - Minimize !
For this, we'll start with an energy minimization to remove steric clashes. 

Usually, two parts follow the minimization:

* A small simulation in the NVT ensemble, with the backbone atoms fixed, while reassigning velocities every xx steps. This is the thermalization step, and can usually be done at once like here, but some more precautious protocols increase the temperature every NVT run until the desired temperature. This is the *mnvt.mdp* parameter file.
* A small simulation in the NPT ensemble. This is where we add pressure to the mix and release the protein. For picky systems, you can release the protein in several steps, with a lower and lower force constant. This is the *mnpt.mdp* parameter file.

** NOTE - THIS SHOULD BE DONE OUTSIDE OF THIS NOTEBOOK **

### Minimize
The last step made sure everything was working fine, topology wise, so we can start to make things move a bit. The first step is the minimization step. We will use an algorithm called *steepest descend* to, as its name hints, get to the closest energy local minima. This allows us to remove any steric clashes and optimize a bit everything.

The parameter file that allows us to do that is minim.mdp. Check it out - you can change *steep* for *cg* to use the conjugate gradients method. As a small comment, the conjugate gradients method is useful only when close to the local energy minima, to have a better precision, and should be done for example prior to normal mode analyses - this is however useless in our case, given the size of the kick we'll apply to our system right after it.

Note - minimizing a structure is **always** a good idea, even without doing anything else with the structure later. You can minimize in a solvated membrane, in water or in vacuo too. 